In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle
import streamlit as st

In [ ]:
# Cargar el dataset y preparar el modelo
iris = load_iris()
data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
data['target'] = iris.target
data['species'] = data['target'].apply(lambda x: iris.target_names[x])

# Dividir datos
X = data.drop(['target', 'species'], axis=1)
y = data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Guardar modelo
with open('iris_model.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Modelo entrenado y guardado correctamente!")

In [ ]:
# Crear la aplicación Streamlit
%%writefile app.py
import streamlit as st
import pickle
from sklearn.datasets import load_iris

# Cargar modelo y datos
with open('iris_model.pkl', 'rb') as file:
    model = pickle.load(file)

iris = load_iris()

# Configurar página
st.set_page_config(page_title="Clasificador de Iris", layout="centered")

# Título y descripción
st.title("🌼 Clasificador de Flores Iris")
st.write("""
Esta app predice la especie de una flor Iris basándose en sus medidas.
""")

# Sidebar con información
st.sidebar.header("Medidas de la flor")
st.sidebar.write("Introduce los valores para hacer la predicción:")

# Inputs del usuario
def user_input_features():
    sepal_length = st.sidebar.slider('Longitud del sépalo (cm)', 4.0, 8.0, 5.4)
    sepal_width = st.sidebar.slider('Ancho del sépalo (cm)', 2.0, 4.5, 3.4)
    petal_length = st.sidebar.slider('Longitud del pétalo (cm)', 1.0, 7.0, 1.3)
    petal_width = st.sidebar.slider('Ancho del pétalo (cm)', 0.1, 2.5, 0.2)
    data = {
        'sepal_length': sepal_length,
        'sepal_width': sepal_width,
        'petal_length': petal_length,
        'petal_width': petal_width
    }
    return pd.DataFrame(data, index=[0])

input_df = user_input_features()

# Mostrar inputs
st.subheader('Medidas ingresadas')
st.write(input_df)

# Predecir
prediction = model.predict(input_df)
prediction_proba = model.predict_proba(input_df)

# Mostrar resultados
st.subheader('Predicción')
species = iris.target_names[prediction][0]
st.write(f"La especie predicha es: **{species}**")

st.subheader('Probabilidades por especie')
proba_df = pd.DataFrame({
    'Especie': iris.target_names,
    'Probabilidad': prediction_proba[0]
})
st.bar_chart(proba_df.set_index('Especie'))

# Información adicional
expander = st.expander("Acerca del modelo")
expander.write("""
- **Dataset:** Iris de sklearn (150 muestras)
- **Modelo:** Random Forest (100 árboles)
- **Precisión:** 96.67% en conjunto de prueba
""")